In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import functions

In [ ]:
# Load data
train = pd.read_parquet('train_final.parquet')  
test = pd.read_parquet('test_final.parquet')

In [ ]:
test.head()

In [ ]:
train = train.drop(['id','carrier','devicebrand'], axis=1)
test_id = test['id']
test = test.drop(['id','carrier','devicebrand'], axis=1)

In [ ]:
# Verilerinizi özellikler ve hedef değişken olarak ayırın
X_train = train.drop(['target'], axis=1)
y_train = train['target']

# Eğitim ve validation setlerini oluşturun
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
# Eğitim ve validation setlerindeki hedef sütununu dönüştürün
y_train_binary = y_train.apply(functions.target_to_binary)
y_valid_binary = y_valid.apply(functions.target_to_binary)


In [ ]:
# Sınıflandırıcıyı oluşturun
clf = ExtraTreesClassifier(n_estimators=100)

# Modeli eğitin
clf.fit(X_train, y_train_binary)

In [ ]:
#Tahminler yapın
y_pred = clf.predict(X_valid)

#y_valid_binary ve y_pred dizilerinin veri türlerini dönüştürün
y_valid_binary = y_valid_binary.astype(np.float64)
y_pred = y_pred.astype(np.float64)


#Jaccard benzerliği hesaplayın
jaccard = functions.jaccard_similarity_score(y_valid_binary, y_pred)

print("Jaccard Skoru:", jaccard)

In [ ]:
# Test verileri üzerinde tahmin yapın
y_test_pred = clf.predict(test)

In [ ]:
# Tahminleri içeren yeni bir veri çerçevesi oluşturun
tahmin_verisi = pd.DataFrame({'id': test_id, 'target': y_test_pred})

# Tahmin verisini CSV dosyasına kaydedin
tahmin_verisi.to_csv('tahminler.csv', index=False, columns=['id', 'target'])